# Sceario Based Model Validation

This dashboard enables scenario based, end-to-end model validation streamlined for the end user.
In this case, model validation means a fully integrated, end-to-end model execution and comparison against expected results.
These comparison are based on well defined scenarios described below.

## Validation Scenarios

The scenarios used for validation can be broken down into two groups: zero cost scenarios, constraint scenarios, and cost comparison scenarios.
Both are described below.

### 1. Zero Cost Scenarios

The zero cost validation scenarios aim to validate that configuring cost driver parameters to zero will lead to zero costs in the output estimates of the models.
Each of the zero cost scenarios is described below.

1. **Zero cost per kwh**: validates that setting electricity costs to zero leads to zero electricity OpEx costs for a school.
    > **Decription:** \
    > **WHEN** the cost per kwh is set to zero for any technology,\
    > **THEN** the electricity opex costs should be zero in the model outputs.

2. **Zero cost internet mbps**: validates that setting mbps cost to zero leads to zero OpEx costs attributed to connectivity for a school.
    > **Decription:** \
    > **WHEN** the mpbs cost is set to zero for any technology,\
    > **THEN** the technology opex costs should be composed of only the maintenance costs in the output.
    
3. **Zero Technology OpEx**: validates that setting maintenance cost and mbps cost to zero leads to zero OpEx costs attributed to technology for a school.
    > **Decription:** \
    > **WHEN** the maintenance cost is set to zero for any technology,\
    > **AND** the mpbs cost is set to zero for any technology,\
    > **THEN** the technology opex costs should be zero in the output.
    
4. **Zero Intallation Cost**: validates that setting installation cost to zero leads to zero CapEx costs attributed to technology installation for a school.
    > **Decription:** \
    > **WHEN** the intallation cost is set to zero for any technology,\
    > **THEN** the technology capex cost should be composed of only the capex attributed to the service provider.
    
5. **Zero Electricity CapEx**: validates that setting solar panel install costs to zero leads to no electricity CapEx costs attributed to a school
    > **Decription:** \
    > **WHEN** the solar panel install cost is set to zero for any technology,\
    > **THEN** the electricity capex is zero.
    
### 2. Technology Constraint Scenarios

These scenarios validate technology constraints relevant to each respective model.
The validation scenarios aim to set the constraint to an extreme to drive the models to a simplified outcome.
Usually, this involved setting the constraint in a way that makes the technology an infeasible connectivity candidate.
Each constraint scenario is described below.

1. **Fiber: zero maximum connection length**: validates that setting the maximum connection length to zero leads to no schools being connected with fiber technology.
    > **Decription:** \
    > **WHEN** modeling fiber technology only,\
    > **AND** the maximum connection length is set to zero,\
    > **THEN** no schools schould be connected using fiber.

2. **Cellular: zero maximum range**: validates that setting the maximum range leads to no schools being connected with cellular technology.
    > **Decription:** \
    > **WHEN** modeling cellular technology only,\
    > **AND** the maximum range is set to zero,\
    > **THEN** no schools schould be connected using cellular.

3. **P2P: zero maximum range**: validates that setting the maximum range leads to no schools being connected with P2P technology.
    > **Decription:** \
    > **WHEN** modeling P2P technology only,\
    > **AND** the maximum range is set to zero,\
    > **THEN** no schools schould be connected using P2P.
    
4. **Demand: exceeds max bandwidth**: validates that setting the bandwidth demand above the bandwidth capacity for a given technology leads to no schools being connected with that technology.
    > **Decription:** \
    > **WHEN** the bandwidth demand exceeds the bandwidth capacity for any technology,\
    > **THEN** no schools schould be connected using that technology.
    
### 3. Single School Scenarios

These scenarios validate non-zero cost drivers on a single school.

1. **Electricity: cost per kwh**: validates that the electricity opex is determined by the power requirement of a technology and the cost per kwh.
    > **Decription:** \
    > **WHEN** estimating cost for a single school that is connected to the grid,\
    > **AND** the annual power requirement for a technology is set to a well-known value of `120` kwh,\
    > **AND** the cost per kwh is set to a well-known value of `1.00` USD per kwH,\
    > **THEN** the annual opex electricity cost at the school should be the product of the two values above of `120` USD,\
    > **AND** the monthly opex electricity cost at the school should be the value above divided by the number of months in a year of `10` USD.
    
2. **Connectivity: cost of mbps**: validates that the connectivity opex is determined by the bandwidth demand and the cost of mbps.
    > **Decription:** \
    > **WHEN** estimating cost for a single school with a given technology of `Satellite LEO`,\
    > **AND** the bandwidth demand requirement for that school is set to a well-known value of `10` mbps,\
    > **AND** the cost per mbps is set to a well-known value of `12.00` USD per mbps,\
    > **THEN** the annual opex connectivity cost at the school is the product of the two values above of `120` USD,\
    > **AND** the monthly opex connectivity cost at the school is the value above divided by the number of months in a year of `10` USD.
    
3. **Fiber: cost per km**: validates that the opex and capex per km of fiber scales with the distance from the nearest node.
    > **Decription:** \
    > **WHEN** estimating the cost of connecting a single school with Fiber,\
    > **AND** the school is a well-known distance from the nearest fiber node of approximately `6.708` km,\
    > **AND** the CapEx cost per km of fiber is a well-known value of `1000` USD per km,\
    > **AND** the OpEx cost per km of fiber is a well-known value of `100` USD per km,\
    > **THEN** the CapEx cost for connecting this school with Fiber is product of the distance and the per km CapEx cost of approximately `6,708` USD,\
    > **AND** the annual OpEx cost for connecting this school with Fiber is product of the distance and the per km OpEx cost of approximately `671` USD.
    
4. **Satellite: installation and maintenance**: validates that the capex and opex for satellite scale with installation and maintenance cost parameters
    > **Decription:** \
    > **WHEN** estimating the cost of connecting a single school with Satellite,\
    > **AND** the installation cost of satellite is a well-known value of `1000` USD,\
    > **AND** the annual maintenance cost of Satellite is a well-known value of `120` USD ,\
    > **THEN** the CapEx cost for connecting this school with Satellite should match the cost of installation above of `1000` USD,\
    > **AND** the annual OpEx cost for connecting this school with Satellite should match the cost of maintenance above of `120` USD.
    
5. **Cellular: installation and maintenance**: validates that the capex and opex for satellite scale with installation and maintenance cost parameters
    > **Decription:** \
    > **WHEN** estimating the cost of connecting a single school with Cellular,\
    > **AND** the installation cost of satellite is a well-known value of `1000` USD,\
    > **AND** the annual maintenance cost of Cellular is a well-known value of `120` USD ,\
    > **THEN** the CapEx cost for connecting this school with Cellular should match the cost of installation above of `1000` USD,\
    > **AND** the annual OpEx cost for connecting this school with Cellular should match the cost of maintenance above of `120` USD.

6. **P2P: install and maintenance 

In [ ]:
import json
from ipywidgets import FileUpload, Output, Label

from giga.viz.notebooks.cost_estimation_parameter_input import CostEstimationParameterInput
from giga.viz.notebooks.parameters.input_parameter import CategoricalDropdownParameter

import warnings
warnings.filterwarnings('ignore')


def uptade_parameters_from_config_file(inputs, file_name):
    with open(file_name, 'r') as f:
        configs = json.load(f)
    inputs.update(configs)
    return inputs


DEFINED_SCENARIOS = {'1.1: Zero Cost per kwh': {'file': 'configs/config_zero_electricity_opex.json',
                                             'scenario_name': 'zero_kwh',
                                             'type': 'full-country'},
                       '1.2: Zero Cost per mbps': {'file': 'configs/config_zero_mbps_opex.json',
                                              'scenario_name': 'zero_mbps',
                                              'type': 'full-country'},
                       '1.3: Zero Tech OpEx': {'file': 'configs/config_zero_tech_opex.json',
                                               'scenario_name': 'zero_tech_opex',
                                               'type': 'full-country'},
                       '1.4: Zero Installation Cost': {'file': 'configs/config_zero_install_cost.json',
                                                       'scenario_name': 'zero_tech_opex',
                                                       'type': 'full-country'},
                       '1.5: Zero Electricity CapEx': {'file': 'configs/config_zero_solar_capex.json',
                                                       'scenario_name': 'zero_solar_capex',
                                                       'type': 'full-country'},
                       '2.1: Fiber: zero maximum connection length': {'file': 'configs/config_constraint_fiber_zero_range.json',
                                                                      'scenario_name': 'constraint_fiber_zero_range',
                                                                      'type': 'full-country'},
                       '2.2: Cellular: zero cell tower range': {'file': 'configs/config_constraint_cellular_zero_range.json',
                                                                'scenario_name': 'constraint_cellular_zero_range',
                                                                'type': 'full-country'},
                       '2.3: P2P: zero range': {'file': 'configs/config_constraint_p2p_zero_range.json',
                                                'scenario_name': 'constraint_p2p_zero_range',
                                                'type': 'full-country'},
                       '2.4: Demand: exceeds max bandwidth': {'file': 'configs/config_constraint_bw_demand.json',
                                                              'scenario_name': 'constraint_bw_demand',
                                                              'type': 'full-country'},
                       '3.1: Electricity: cost per kwh': {'file': 'configs/config_single_school_electricity_opex.json',
                                                           'scenario_name': 'single_school_electricity_opex',
                                                           'type': 'select-schools', # has electricity
                                                           'school_ids': ['c69b3c2a-d9ed-4ee8-bafc-5a952baacfcb']},
                       '3.2: Connectivity: cost of mbps': {'file': 'configs/config_single_school_connectivity_opex.json',
                                                           'scenario_name': 'single_school_conn_opex',
                                                           'type': 'select-schools',
                                                           'school_ids': ['c69b3c2a-d9ed-4ee8-bafc-5a952baacfcb']},
                       '3.3: Fiber: cost per km': {'file': 'configs/config_single_school_fiber_cost.json',
                                                           'scenario_name': 'single_school_fiber_cost',
                                                           'type': 'select-schools',
                                                           'school_ids': ['77fdcab0-f210-4916-be28-cc6a245498d5']},
                       '3.4: Satellite: install and maintenance': {'file': 'configs/config_single_school_satellite_cost.json',
                                                                 'scenario_name': 'single_school_satellite_cost',
                                                                 'type': 'select-schools',
                                                                 'school_ids': ['77fdcab0-f210-4916-be28-cc6a245498d5']},
                       '3.5: Cellular: install and maintenance': {'file': 'configs/config_single_school_cellular_cost.json',
                                                                'scenario_name': 'single_school_cellular_cost',
                                                                'type': 'select-schools',
                                                                'school_ids': ['77fdcab0-f210-4916-be28-cc6a245498d5']}
                      }

options = list(DEFINED_SCENARIOS.keys())
value = options[0]
description = 'Validation Scenario:'

p = CategoricalDropdownParameter(options=['Default'] + options,
                             value='Default',
                             description=description,
                             parameter_type='categorical_dropdown').parameter

inputs = CostEstimationParameterInput(local_data_workspace='workspace')

output = Output()

def on_change(change):
    with output:
        global inputs 
        inputs = uptade_parameters_from_config_file(inputs, DEFINED_SCENARIOS[p.value]['file'])
        output.clear_output()
    display(p, inputs.parameter_input(), output)

p.observe(on_change, names='value')
display(p, inputs.parameter_input(), output)

In [ ]:
from IPython.display import display, clear_output
from ipywidgets import Button, Output
from tqdm import tqdm
from IPython.display import clear_output
import sys
import time
import logging
import geopandas as gpd

from giga.models.scenarios.scenario_dispatcher import create_scenario
from giga.data.space.model_data_space import ModelDataSpace
from giga.schemas.output import OutputSpace
from giga.viz.notebooks.helpers import run_message
from giga.utils.logging import LOGGER
from giga.viz.notebooks.helpers import download_link_scenario_config
from giga.viz.notebooks.tables import display_summary_table, plot_cost_breakdown
from giga.viz.notebooks.helpers import output_to_table, download_link_frame

def filter_schools(data_space, scenario):
    if scenario['type'] == 'select-schools':
        # filter
        return data_space.filter_schools(scenario['school_ids'])
    else:
        return data_space

button = Button(description="Run Model")
output = Output()

output_space = None
data_space = None
config = None

display(button, output)

def on_button_clicked(b):
    with output:
        clear_output(wait=True)
        
        global config
        global data_space
        global output_space
        
        selected_scenario = DEFINED_SCENARIOS[p.value]
        
        verbose = inputs.dashboard_parameters()["verbose"]
        LOGGER.propagate = verbose
        
        config = inputs.scenario_parameters()
        data_space = ModelDataSpace(inputs.data_parameters())
        data_space = filter_schools(data_space, selected_scenario)
        output_space = OutputSpace()
        scenario = create_scenario(config, data_space, output_space)
        
        output_space = scenario.run(progress_bar=verbose)
        run_message()
        
        display(download_link_scenario_config(inputs))
        
        display_summary_table(output_space, data_space)
        border_file = 'parameter_workspace/TM_WORLD_BORDERS-0.3/TM_WORLD_BORDERS-0.3.shp'
        borders = gpd.read_file(border_file)
        borders['NAME'] = borders['NAME'].apply(lambda x: x.lower())
        cid = inputs.data_parameters().school_data_conf.country_id
        country_id = 'rwanda' if cid == 'sample' else cid
        b = borders[borders["NAME"] == country_id]
        plot_cost_breakdown(output_space, data_space, b)
        
        table = output_to_table(output_space)
        display(download_link_frame(table))

button.on_click(on_button_clicked)

In [ ]:
import geopandas as gpd
from ipywidgets import HTML

from giga.viz.notebooks.tables import display_summary_table, output_to_capex_details, plot_cost_breakdown
from giga.viz.notebooks.helpers import output_to_table, button_cb, download_link_frame
from giga.viz.notebooks.maps import show_cost_map, show_electricity_map
from giga.viz.notebooks.fiber import plot_fiber_connections, default_rwanda_map


def action():
    
    if inputs.data_parameters().school_data_conf.country_id == 'brazil':
        display(HTML("<hr><b><font color='#5b8ff0'>Unable to display maps for Brazil</b><br><br>"))
        return
    
    display(HTML("<hr><b><font color='#5b8ff0'>Electricity Map</b><br><br>"))
    display(show_electricity_map(data_space))
    
    display(HTML("<hr><b><font color='#5b8ff0'>Cost Map</b><br><br>"))
    display(show_cost_map(data_space, output_space))
    
    if output_space.fiber_costs is not None:
        display(HTML("<hr><b><font color='#5b8ff0'>Fiber Map</b><br><br>"))
        display(plot_fiber_connections(data_space.fiber_coordinates, data_space.school_coordinates, output_space.fiber_costs.technology_results.distances, m=default_rwanda_map()))
        
    
button, click = button_cb("Show Maps", action)
button.on_click(click)